In [ ]:
import glob
from pyhdf.SD import SD, SDC
import numpy as np
import math
from __future__ import print_function
import matplotlib
matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime
import subprocess
from numpy import genfromtxt
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
import time
import numexpr as ne
import bottleneck as bn
from numpy import *
import pandas as pd

In [ ]:
import plotly.plotly as py
plotly.tools.set_credentials_file(username='Kshitij123', api_key='XT7SgVCwq3GFDxyu8K7G')
import plotly.

In [ ]:
file = SD('/panfs/roc/groups/6/kumarv/tayal007/MODIS_TILES/MODIS_h11v04/MOD09A1.A2000049.h11v04.006.2015136143357.hdf', SDC.READ)
print (file.info())

In [ ]:
datasets_dic = file.datasets()
for idx,sds in enumerate(datasets_dic.keys()):
    print (idx, sds)

In [ ]:
sds_obj = file.select('sur_refl_b07') 
#sds_obj = file.select('sur_refl_b01')  # select sds
data = np.array(sds_obj.get()).astype(np.float32) # get sds data
print (data.shape)
print (data)
#plt.imshow(data)
#plt.show()

In [ ]:
p1 = np.array([[5,6]])
p2 = np.array([[4,7,9]])
p1.shape
p2.shape
Q= np.concatenate((p1,p2),axis=1)
print(Q)
#Q.shape

In [ ]:
data.flatten()
print (data.shape)
print (type(data))

In [ ]:
a = np.array([[1,2], [3,4]])
b = np.array([[7,8], [0,1]])
Q= np.concatenate((a.flatten(),b.flatten(), b.flatten()),axis=0)
print (Q)
print (Q.shape)

In [ ]:
def standardize (data, Quality):
    data.astype(np.float32)[np.logical_or(Quality!= 0,data==-28672)] = np.nan
    data = np.divide((data - bn.nanmean(data)), bn.nanstd(data))
    return data.astype(np.float16).flatten();

In [ ]:
import pprint
pprint.pprint(sds_obj.attributes())

In [ ]:
print (np.count_nonzero(data>16000))
print (np.count_nonzero(data==-28672))
print (np.count_nonzero(data<-100))
print (np.count_nonzero(data==3))
print (np.count_nonzero(data==4))
print (np.count_nonzero(data==5))
print (np.count_nonzero(data==6))
print (np.count_nonzero(data==7))
print (np.count_nonzero(data==8))
print (np.count_nonzero(data==12))

#plt.hist(data.tolist(), bins='auto')  # arguments are passed to np.histogram

In [ ]:
mask = 

In [ ]:
quality_obj = file.select('sur_refl_state_500m')
quality = quality_obj.get()
# x = np.vectorize(np.binary_repr)(quality)
v


In [ ]:
np.count_nonzero(p == 0)
NDVIaverage[p != 0] = np.nan
print (NDVIaverage)

In [ ]:
import pprint
pprint.pprint( (sds_obj.attributes()))

In [ ]:
def Timeseriesmatrix(directorypath, sorteddates):
    #NDVI_CON = np.zeros(shape=(2400,2400,len(sorteddates)),dtype=np.float32)
    i = 0
    dict1 = {}
    for k, v in sorted(sorteddates.items()):
        start = time.time()
        matching = [s for s in directorypath if v in s]
#         print (matching[0])
        try:
            file = SD(matching[0], SDC.READ)
            sds_obj01 = file.select('sur_refl_b01')  # select sds Red
            sds_obj02 = file.select('sur_refl_b02')  # select  sds NIR
            sds_obj03 = file.select('sur_refl_b03')  # select sds Red
            sds_obj04 = file.select('sur_refl_b04')  # select  sds NIR
            sds_obj05 = file.select('sur_refl_b05')  # select sds Red
            sds_obj06 = file.select('sur_refl_b06')  # select  sds NIR
            sds_obj07 = file.select('sur_refl_b07')  # select  sds NIR
            sds_obj08 = file.select('sur_refl_state_500m') # select quality sds

            arr1 = np.array(sds_obj01.get()) # get sds data
            arr2 = np.array(sds_obj02.get())
            arr3 = np.array(sds_obj03.get()) # get sds data
            arr4 = np.array(sds_obj04.get())
            arr5 = np.array(sds_obj05.get()) # get sds data
            arr6 = np.array(sds_obj06.get()) # get sds data
            arr7 = np.array(sds_obj07.get()) 
            Quality = np.array(sds_obj08.get())
            Quality = Quality & 3

            dict1[k] = np.concatenate((standardize(arr1,Quality),standardize(arr2,Quality), standardize(arr3,Quality),standardize(arr4,Quality),standardize(arr5,Quality),standardize(arr6,Quality),standardize(arr7,Quality)),axis=0)

            end = time.time()
            print (i, (end - start))
            #if (i==5):break;
            i = i + 1
        except:
            print (matching[0])
    return dict1

def sortedtimestamp(directorypath):
   timestamps = dict()
   for path in directorypath:
        timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
   return timestamps;

In [ ]:
directorypath = glob.glob("/home/kumarv/tayal007/MODIS_TILES/MODIS_h11v04/*.hdf");
# print (directorypath)
sorteddates = sortedtimestamp(directorypath)
dict1 = Timeseriesmatrix(directorypath, sorteddates);
#print ("Final NDVI shape is " + str(NDVISTACK.shape));
#NDVIaverage = np.nanmean(NDVISTACK, axis=2);
#np.save('/home/kumarv/tayal007/MODIS_TILES/h21v07', NDVISTACK)
#np.save('/home/kumarv/tayal007/CALIFORNIA1_CLOUDFILTER',NDVIaverage )

In [ ]:
import pickle
output = open('minnesota.pkl', 'wb')
pickle.dump(dict1, output)
output.close()

In [ ]:
dict1 = None

In [ ]:
Q = pd.DataFrame.from_dict(dict1)
Q.head()

In [ ]:
Q.to_hdf('data.h5', key='Q', mode='w')

In [ ]:
Q.to_pickle("minnesota.pkl")

In [ ]:
Q = pd.read_pickle("minnesota.pkl")

### NDVISTACK = np.load('/home/kumarv/tayal007/NDVISTACK_CLOUDFILTER.npy')

In [ ]:
p1 = np.array([[5,6]])
p2 = np.array([[4,7,9]])
p1.shape
p2.shape
Q= np.concatenate((p1,p2),axis=1)
print(Q)
#Q.shape

In [ ]:
print (len(sorteddates))

In [ ]:
print (len(sorteddates))
dates = []
for k, v in sorted(sorteddates.items()):
    dates.append(k);

In [ ]:
for i,n in enumerate(dates):
    if (n.year==2013):
        print (i)

In [ ]:
#NDVISTACK = np.load('/home/kumarv/tayal007/CALIFORNIA1_CLOUDFILTER.npy')
#NDVIaverage = np.load('/home/kumarv/tayal007/NDVIaverage.npy')

In [ ]:
def percentage(part, shape):
    whole = shape[0] * shape[1] * shape[2]
    return 100 * float(part)/float(whole)

In [ ]:
print ("Missing value in raw data is ", np.count_nonzero(np.isnan(NDVISTACK)),'which is ' ,percentage(np.count_nonzero(np.isnan(NDVISTACK)),NDVISTACK.shape),'%')

In [ ]:
print (NDVISTACK.shape)
print (np.count_nonzero(np.isnan(NDVISTACK)))

In [ ]:
print (np.count_nonzero(np.isinf(NDVISTACK)))

In [ ]:
from numpy import inf
NDVISTACK1 = NDVISTACK
NDVISTACK1[NDVISTACK == inf] = np.nan
NDVISTACK1[NDVISTACK == -inf] = np.nan
print (np.count_nonzero(np.isinf(NDVISTACK1)))

In [ ]:
print (np.count_nonzero(NDVISTACK1>1))
print (np.count_nonzero(NDVISTACK1<-1))
NDVISTACK1[NDVISTACK1 > 1] = np.nan
NDVISTACK1[NDVISTACK1 < -1] = np.nan

In [ ]:
#timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
Years = []
for x in dates:
    Years.append(x.year)
UniqueYears = sorted(list(set(Years)))
Years = []
for x in UniqueYears:
   print (str(x))
   Years.append(datetime.datetime.strptime('30/12/' + str(x), '%d/%m/%Y'))
print (Years)

In [ ]:
np.save('/home/kumarv/tayal007/ContiChange', ContiChange)

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

six_months = date.today() + relativedelta(months=-12)
print (six_months)

NO_NEG_CLEANED = np.copy(CLEANED_DATA)
NO_NEG_CLEANED[CLEANED_DATA < 0] = 0
ContiChange = np.zeros(shape=(2400,2400,len(dates[48:-48])),dtype=np.float32)
print (len(dates[48:-48]))
for index,l in enumerate((dates[48:-48])):
    start = time.time()
    prev12_months = l + relativedelta(months=-12)
    forw12_months = l + relativedelta(months=+12)
    for i,j in enumerate(dates):
        if j >= prev12_months:
            l1 = i
            break
    for i,j in enumerate(dates):
        if j >= forw12_months:
            l2 = i
            break
    ContiChange[:,:,index] = np.absolute(np.nanmean(NO_NEG_CLEANED[:,:,l1:index+48], axis=2) - np.nanmean(NO_NEG_CLEANED[:,:,index+48:l2], axis=2)).astype(np.float32);
    end = time.time()
    print(index, end - start)

In [ ]:
YearChange = np.zeros(shape=(2400,2400,len(UniqueYears)),dtype=np.float32)
print (YearChange.shape)
i = 0
old = 0
new = 0
for x in dates:
    if x.year == UniqueYears[i]:
        new = new + 1
    else:
        mini = CLEANED_DATA[:,:,old:new]
        mini[mini < 0] = 0;
        YearChange[:,:,i] = (np.nanmean(mini, axis=2)).astype(np.float32);
        i = i + 1
        old = new
        new = new + 1
mini = CLEANED_DATA[:,:,old:new]
mini[mini < 0] = 0;
YearChange[:,:,i] = (np.nanmean(mini, axis=2)).astype(np.float32);

In [ ]:
YearYearChange = np.zeros(shape=(2400,2400,(len(UniqueYears) - 3)),dtype=np.float32)
for j in range(len(UniqueYears) - 3):
    YearYearChange[:,:,j] = np.absolute(YearChange[:,:,(j + 2)] - YearChange[:,:,j+1])
YearYearChange_average = np.nanmax(YearYearChange, axis=2);

In [ ]:
s = 'data!=0,data!=7'
leval(s)

In [ ]:
print (data!=0)
print (np.logical_and.reduce((data!=0)))

In [ ]:
Filter = "0,7"
p = Filter.split(',')
mask = np.copy(YearYearChange_average)
s = 'data!=0'
print (type(s))
print (data)
print (mask)
mask[np.logical_and.reduce((data!=0))] = 0
print (mask)

In [ ]:
print (np.count_nonzero(YearYearChange_average>0))
print ((mask>0))

In [ ]:
p = 5
def top_n_indexes(arr, n):
    idx = bn.argpartition(arr, arr.size-n, axis=None)[-n:]
    width = arr.shape[1]
    return [divmod(i, width) for i in idx]

def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

print (mask.shape)
idx = top_n_indexes(mask, 20)
#idx = largest_indices(mask,10)
# idx.sort(key = lambda tup: tup[0])
idx.sort(key = lambda tup: mask[tup], reverse=True)
print (idx)
lo = ''
for x in idx:
    lo = lo + "{lat:" + str(x[0]) + ', lng: ' + str(x[1]) + '},'
    print (mask[x])
print ("B")
for x in idx[5:]:
    lo = lo + "{lat:" + str(x[0]) + ', lng: ' + str(x[1]) + '},'
    print (mask[x])

In [ ]:
MaxMin = np.zeros(shape=(2400,2400,len(UniqueYears)),dtype=np.float32)
print (MaxMin.shape)
i = 0
old = 0
new = 0
for x in dates:
    if x.year == UniqueYears[i]:
        new = new + 1
    else:
        mini = np.nanmin(CLEANED_DATA[:,:,old:new], axis=2)
        mini[mini < 0] = 0;
        MaxMin[:,:,i] = (np.nanmax(CLEANED_DATA[:,:,old:new], axis=2) - mini).astype(np.float32);
        i = i + 1
        old = new
        new = new + 1
mini = np.nanmin(CLEANED_DATA[:,:,old:new], axis=2)
mini[mini < 0] = 0;
MaxMin[:,:,i] = (np.nanmax(CLEANED_DATA[:,:,old:new], axis=2) - mini).astype(np.float32);

In [ ]:
MaxMin_average = np.nanmean(MaxMin, axis=2);

In [ ]:
print (MaxMin_average[144,1402])

In [ ]:
idx = top_n_indexes(mask, 10)
idx.sort(key = lambda tup: tup[0])
print (idx)
lo = ''
#{lat: -31.563910, lng: 147.154312},
#{lat: -33.718234, lng: 150.363181},
#for x in idx:
# print (MaxMin_average[x])
for x in idx:
   lo = lo + "{lat:" + str(x[0]) + ', lng: ' + str(x[1]) + '},'
   print (MaxMin_average[x])
#print (lo)

In [ ]:
np.set_printoptions(precision=2)
print (np.array_str(CLEANED_DATA[255,1295,:], precision=2, suppress_small=True))
print (np.array_str(MaxMin[255,1295,:], precision=2, suppress_small=True))

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(NDVIaverage)
while True:
    x = plt.ginput(1)
#     print (NDVISTACK[int(round(x[0][1])),int(round(x[0][0])),:])
    print("clicked", x)
    #lat = latitude[int(round(x[0][0]))][int(round(x[0][1]))]
    #long = longitude[int(round(x[0][0]))][int(round(x[0][1]))]
    #print ('http://maps.google.com/?q=' + str(lat) + ',' + str(long))
    showtimeseries(int(round(x[0][0])), int(round(x[0][1])));
    
plt.show()

In [ ]:
def showtimeseries(yvalue, xvalue):  
#     namesnan = sorteddates;
#     countsnan = NDVISTACK[xvalue,yvalue,:]
#     assert(len(namesnan) == countsnan.shape[0])
#     names = []
#     counts = []
#     for x in range(len(namesnan)):
#         if  math.isnan(countsnan[x]) == False:
#                 names.append(namesnan[x]);
#                 counts.append(countsnan[x]);
    names = sorteddates
    counts = NDVISTACK[xvalue,yvalue,:]
    plt.subplot(1, 2, 2).cla()
    plt.subplot(1, 2, 2)
    pylab.figure(1)
    axes = plt.gca()
    axes.set_ylim([0,1])
    x = range(len(names))
    pylab.xticks(x, names)
    plt.xticks( rotation=90,fontsize=5)
    pylab.plot(x,counts,"g")
    plt.plot(x,counts, 'ro')
    plt.tight_layout()

In [ ]:
latitude = genfromtxt('/home/kumarv/tayal007/latitide.csv', delimiter=',')
longitude = genfromtxt('/home/kumarv/tayal007/longitude.csv', delimiter=',')

In [ ]:
def showtimeseries12(yvalue, xvalue):  
    namesnan = dates;
    countsnan = NDVISTACK[xvalue,yvalue,:]
    assert(len(namesnan) == countsnan.shape[0])
    names = []
    counts = []
    for x in range(len(namesnan)):
        if  math.isnan(countsnan[x]) == False:
                names.append(namesnan[x]);
                counts.append(countsnan[x]);
    assert(len(names) == len(counts))
    data = [go.Scatter(x=names, y=counts)]
    plotly.offline.iplot(data)

In [ ]:
from IPython.display import clear_output
plt.imshow(NDVIaverage)
while True:
    x = plt.ginput(1)
    clear_output()
    try:
#         lat = latitude[int(round(x[0][0]))][int(round(x[0][1]))]
#         long = longitude[int(round(x[0][0]))][int(round(x[0][1]))]
#         print ('http://maps.google.com/?q=' + str(lat) + ',' + str(long))
        showtimeseries(int(round(x[0][0])), int(round(x[0][1])));
    except IndexError:
        print ()

In [ ]:
from scipy.signal import lfilter
import itertools
from itertools import compress
def showtimeseries(xvalue, yvalue):  
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    
    iA = ~np.isnan(CLEANED_DATA[xvalue,yvalue,:])
    NonNanList = np.where(iA == True)
    Nandates = list(compress(dates, iA))


#     namesnan = dates;
#     countsnan = NDVISTACK[xvalue,yvalue,:]
#     assert(len(namesnan) == countsnan.shape[0])
#     names = []
#     counts = []
#     counts2 = NDVISTACK[xvalue,yvalue,:][~np.isnan(NDVISTACK[xvalue,yvalue,:])]
#     for x in range(len(namesnan)):
#         if  math.isnan(countsnan[x]) == False:
#                 names.append(namesnan[x]);
#                 counts.append(countsnan[x]);
#     yy = lfilter(b,a,counts)
#     print (len(Nandates))
    data1 = go.Scatter(x=Nandates[5:], y=list(itertools.chain.from_iterable(CLEANED_DATA[xvalue,yvalue,NonNanList]))[5:],name = 'MODIS')
  #  data2 = go.Scatter(x=names[5:], y=yy[5:],name = 'FILTERED')
    #data3 = go.Scatter(x=names[5:], y=counts[5:],name = 'test')
    #data3 = go.Scatter(x=names[5:], y=counts2[5:],name = 'FILTERED_TEST')
#     print (type(Nandates))
    #data1 = go.Scatter(x=Nandates, y=NDVISTACK[xvalue,yvalue,NonNanList],name = 'MODIS')
    layout = go.Layout(
    title = 'NDVI Plot', 
    width = 1800,
    height = 900,
    xaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        tickangle=-30,
        tick0=datetime.datetime.strptime('2000/12/30', '%Y/%m/%d'),
        dtick= 'M12',
        mirror='ticks',
        gridcolor='#000000',
        gridwidth=1,
        zerolinecolor='#000000',
        zerolinewidth=2,
        linecolor='#000000',
        linewidth=4,
        tickwidth=2,
        tickformat = "%Y",
        hoverformat = "%d-%b-%Y"
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=False,
        mirror='ticks',
        gridcolor='#000000',
        gridwidth=1,
        zerolinecolor='#000000',
        zerolinewidth=2,
        linecolor='#000000',
        linewidth=4
    )
)
    data = [data1]
    fig = go.Figure(data=data, layout=layout)
    print (len(list(itertools.chain.from_iterable(CLEANED_DATA[xvalue,yvalue,NonNanList]))))
    plotly.offline.iplot(fig)

In [ ]:
showtimeseries(105,5)

In [ ]:
print (CLEANED_DATA[0,1,:])

In [ ]:
CLEANED_DATA = np.zeros(shape=(2400,2400,len(dates)),dtype=np.float32)

In [ ]:
def computation():
    global CLEANED_DATA
    n = 8  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    for i in range(2400):
        start = time.time()
        for j in range(2400):
            timeseries = NDVISTACK1[i,j,:]
            #heights = timeseries[~np.isnan(timeseries)]
            yy = lfilter(b,a,timeseries[~np.isnan(timeseries)])
            index = 0
            for x in range(len(dates)):
                if  np.isnan(timeseries[x]):
                    CLEANED_DATA[i,j,x] = np.nan
                else:
                    CLEANED_DATA[i,j,x] = yy[index]
                    index = index + 1
        end = time.time()
        print ("Completed " + str(i) + " " + str(end - start))            

        
def parallelcomputation(start_x,start_y,interval):
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    for i in range(interval):
        for j in range(interval):
            timeseries = NDVISTACK1[start_x + i,start_y + j,:]
            yy = lfilter(b,a,timeseries[~np.isnan(timeseries)])
            index = 0
            for x in range(len(dates)):
                if  np.isnan(timeseries[x]):
                    CLEANED_DATA[i,j,x] = np.nan
                else:
                    CLEANED_DATA[i,j,x] = yy[index]
                    index = index + 1
        print ("Completed ", start_x, start_y, i )
    return ""

In [ ]:
computation()
np.save('/home/kumarv/tayal007/cleaned_data', CLEANED_DATA)

In [ ]:
CLEANED_DATA = np.load('/home/kumarv/tayal007/cleaned_data.npy')

In [ ]:
start = time.time()
nprocs = []   # saves the process
for item in threadlist:                      #Specify number of thread
    n = multiprocessing.Process(target=parallelcomputation, args=(item[0],item[1],400))   # multiprocessing 
    nprocs.append(n)
    n.start()
        
for i in nprocs:   
    i.join()        # waiting for all the process to finish
end = time.time()
print ((end - start))

In [ ]:
import multiprocessing
threadlist = [[0,0],[0,400],[0,800],[0,1200],[0,1600],[0,2000],[400,0],[400,400],[400,800],[400,1200],[400,1600],[400,2000],[800,0],[800,400],[800,800],[800,1200],[800,1600],[800,2000],[1200,0],[1200,400],[1200,800],[1200,1200],[1200,1600],[1200,2000],[1600,0],[1600,400],[1600,800],[1600,1200],[1600,1600],[1600,2000],[2000,0],[2000,400],[2000,800],[2000,1200],[2000,1600],[2000,2000]]
print (threadlist)

In [ ]:
print (CLEANED_DATA)

In [ ]:
p = 0
def ksh():
    global p
    p = 5

In [ ]:
ksh()
print (p)

In [ ]:
p = np.array([5,6,np.nan])

In [ ]:
for a,b in enumerate(p):
    print(a,b)

In [ ]:
iA = ~np.isnan(p);
print (iA)
np.where(iA == True)

In [ ]:
print (NDVISTACK[2,3,:])
iA = ~np.isnan(NDVISTACK[2,3,:]);
NonNanList = np.where(iA == True)
print (NDVISTACK[2,3,NonNanList])

In [ ]:
from itertools import compress
print (len(dates))
list_a_np = np.array(dates)
print (list_a_np[iA].shape)
len(list(compress(dates, iA)))

In [ ]:
p = [5,6,7]
q = np.array(p)
print (p[(0,1)])

In [ ]:
from numpy import genfromtxt
latitude = genfromtxt('/home/kumarv/tayal007/LongLatCoordinates/h08v05/latitide.csv', delimiter=',')
longitude = genfromtxt('/home/kumarv/tayal007/LongLatCoordinates/h08v05/longitude.csv', delimiter=',')


In [ ]:
print (latitude[180,1409],longitude[180,1409], precision=8)

In [ ]:
np.set_printoptions(precision=10)
print (latitude[180,1408],longitude[180,1408])

In [ ]:
print (latitude[180,1410],longitude[180,1410])

In [ ]:
k = 'https://www.croplands.org/app/map?lat=11.56862&lng=32.70421743392945&zoom=16'

In [ ]:
k.split('/')

In [ ]:
k

In [ ]:
lat = float(k.split('/')[4].split('&')[0].split('=')[-1])
lng = float(k.split('/')[4].split('&')[1].split('=')[1])
zoom = int(k.split('/')[4].split('&')[2].split('=')[1])

In [ ]:
zoom

In [ ]:
int(zoom.split('=')[1])

In [ ]:
float(lng.split('=')[1])

In [ ]:
float(lat.split('=')[-1])

In [ ]:
croplandurl = 'a'

In [ ]:
i = 2

In [ ]:
lat = 5
if ( i == 2):
    lat = 6

In [ ]:
lat